In [8]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType

In [9]:
spark = SparkSession\
    .builder\
    .appName("Solver problema 1")\
    .getOrCreate()

In [10]:
# Para esta primera parte basta con inferencia para extraer la data necesaria.
file = "../tests/vshort-users-details-2023.csv"

df = spark\
	.read\
	.format("csv")\
	.option("header", "true")\
	.load(file)

df.show(5)

+------+--------+------+--------------------+--------------------+--------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|Mal ID|Username|Gender|            Birthday|            Location|              Joined|Days Watched|Mean Score|Watching|Completed|On Hold|Dropped|Plan to Watch|Total Entries|Rewatched|Episodes Watched|
+------+--------+------+--------------------+--------------------+--------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|     1|   Xinil|  Male|1985-03-04T00:00:...|          California|2004-11-05T00:00:...|       142.3|      7.37|     1.0|    233.0|    8.0|   93.0|         64.0|        399.0|     60.0|          8458.0|
|     3| Aokaado|  Male|                NULL|        Oslo, Norway|2004-11-11T00:00:...|        68.6|      7.34|    23.0|    137.0|   99.0|   44.0|         40.0|        343.0|     15.0|        

In [14]:
# Se genera la VIEW
df.createOrReplaceTempView("df")

In [15]:
# Por DATAFRAME
df.select("Mal ID", "Location").show(5)

+------+--------------------+
|Mal ID|            Location|
+------+--------------------+
|     1|          California|
|     3|        Oslo, Norway|
|     4|Melbourne, Australia|
|     9|                NULL|
|    18|                NULL|
+------+--------------------+
only showing top 5 rows



In [17]:
# Por SQL
spark.sql("SELECT 'Mal ID', Location FROM df").show(5)


+------+--------------------+
|Mal ID|            Location|
+------+--------------------+
|Mal ID|          California|
|Mal ID|        Oslo, Norway|
|Mal ID|Melbourne, Australia|
|Mal ID|                NULL|
|Mal ID|                NULL|
+------+--------------------+
only showing top 5 rows



In [7]:
spark.stop()